In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

path = '/content/drive/MyDrive/NNFL Assignment - 2/Datasets/data5.csv'
data = pd.read_csv(path)

data.head()

,Unnamed: 0,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57,58,59,60,61,62,63,64,65,66,67,68,69,70,71,72
0,0,28641.397310,34948.49934,3847.886139,30885.68295,7481.862978,12289.116600,23057.723730,91288.36530,90712.97016,155381.67640,241721.89530,161046.52890,16211.364000,13558.012790,858.977303,14675.18030,5141.236350,3166.781914,17902.272630,68598.216110,90309.56151,140390.179200,168642.182900,84613.593820,4525.830605,7961.894100,3532.734456,5363.993852,2034.490569,4621.747329,13191.002970,77494.871590,71907.065870,75891.065440,61003.998920,24680.621900,255.064515,451.158499,272.216205,284.958998,150.309755,297.201210,565.510507,3143.260472,2075.553186,2981.372334,3497.213672,1257.335645,8.612608,2.586302,0.763240,0.018955,0.002570,0.000974,8.602813,3.038620,0.338269,0.004324,0.000687,0.000249,6.336754,5.487103,0.128553,0.027852,0.004050,0.000585,4.904893,3.183134,2.391377,0.997496,0.410240,0.067070,0.0
1,1,11716.791410,38816.36954,8735.337066,23089.07968,521.447299,20851.327990,6028.821828,12450.46492,40658.61100,58388.47184,49030.53871,26227.47721,343.897579,712.554921,147.283708,491.34558,67.680616,343.249985,248.096039,94.936152,471.93694,770.907607,773.910337,520.954571,35.883775,31.878131,66.228830,17.504204,43.053141,40.124587,28.489483,31.614778,20.233456,15.340029,13.914535,13.481195,23.180416,130.481043,76.388843,57.578184,17.032306,97.905649,12.243984,37.756053,39.026009,32.034927,32.609026,13.850034,10.461923,1.263117,0.196165,0.052477,0.010018,0.000717,9.369977,1.833004,0.460712,0.304236,0.010340,0.004330,5.459352,3.871804,1.988910,0.348642,0.113074,0.090181,6.957099,3.047312,0.992426,0.557659,0.203299,0.089419,1.0
2,2,27560.638120,39821.30075,38081.061940,24186.56173,22828.284870,32054.263710,16491.540070,204319.40510,225918.72420,133021.03120,100797.23530,37039.31523,9018.948076,35834.011140,30198.022380,14884.09426,10638.284370,30762.931230,10289.870720,113652.957900,130637.02580,86856.702560,57588.955070,17394.083840,1537.389978,60932.007690,68010.220480,14227.572940,19539.323660,64101.962140,3582.668029,32278.148740,21767.879630,22716.427360,15942.337630,10399.601130,1200.254838,1033.731312,3910.319962,140.513650,2298.679377,2171.300878,486.980193,7875.535995,11627.903900,7340.450177,5291.619036,1301.369888,7.179882,4.599445,0.161153,0.036392,0.006966,0.000781,7.172213,4.627815,0.169937,0.013939,0.000776,0.000237,8.058336,3.723582,0.180971,0.014862,0.003808,0.002719,6.164694,3.897421,1.356026,0.387332,0.113820,0.041646,1.0
3,3,12837.096380,52503.95848,26907.090880,25929.47630,6747.068355,36500.908160,17448.763790,30427.68767,14560.68952,43987.69096,31511.27172,51232.01613,5108.042827,34479.816870,34597.838710,11137.79128,11228.165220,33264.979740,18839.538430,52298.530370,15137.17750,14205.027090,10353.270880,16567.692060,3181.125978,34319.914850,20488.591250,13605.416160,3253.340518,26617.932320,5514.729309,12289.026460,3679.509072,20133.787560,15386.611930,25574.448520,726.728697,726.294886,1614.950492,323.093919,989.018424,989.276024,338.825690,944.403332,1358.968327,1857.770723,625.331453,1298.337899,8.649521,3.005141,0.178861,0.119111,0.026632,0.004829,7.576529,4.301029,0.087710,0.016552,0.002113,0.000717,10.361394,1.492121,0.104239,0.017744,0.008080,0.001064,4.499628,3.124439,2.628238,1.238560,0.291926,0.132314,1.0
4,4,9292.629213,13424.37494,4145.234774,10320.88592,3362.544079,6461.011953,24535.417660,95682.30252,158949.47450,144837.83750,96264.29882,47009.27200,21010.073480,12758.864880,1456.907239,16543.14959,8058.100577,1847.877739,9302.545210,63116.810140,146587.44080,214544.960600,210626.379000,129394.108100,4556.530140,3733.393694,555.115533,4013.194169,1629.368256,1004.664951,1667.107332,17920.685220,40503.843080,54961.746820,47074.837950,26265.493170,2008.818644,1443.590827,165.475448,1685.990751,728.354092,302.099296,119.984020,4671.127226,13460.613640,21189.609230,19795.527310,

In [ ]:
# normalization
def normalize(x):
  xmean = np.mean(x, axis=0)
  xstd = np.std(x)
  ret = (x - xmean)/(xstd)  
  return ret

In [ ]:
data = np.array(data)
N, M = data.shape
data = data[:, 1:]

np.random.shuffle(data)
N, M = data.shape
ones = np.ones((N, 1))


xnorm = data[:, :M-1]
xnorm = normalize(xnorm)


X = xnorm
X = np.concatenate((ones, X), axis=1)
Y = data[:, M-1:]

In [ ]:
# kmeans
def get_kmeans(k, iters):
  kmeans = X[:k, :]
  clusters = []
  counts = []

  for i in range(k):
    clusters.append(0)
    counts.append(0)


  for c in range(iters):
    for i in range(X.shape[0]):
      point = X[i:i+1, :]
      distances = []
      for j in range(k):
        dis = np.linalg.norm(point - kmeans[j:j+1, :])
        distances.append(dis)

      clus = distances.index(min(distances))
      clusters[clus] = clusters[clus] + min(distances)
      counts[clus] = counts[clus] + 1
    
    for i in range(k):
      clusters[i] = clusters[i] / counts[i]
      kmeans[0][i] = clusters[i]

  return kmeans

In [ ]:
def rad_basis_fn(x, mu, func='gauss'):
  if (func == 'linear'):
    temp = np.linalg.norm(x - mu)
    return temp
  
  elif (func == 'quad'):
    t1 = np.linalg.norm(x - mu)
    t1 = t1**2
    t2 = np.std(x)

    ret = (t1 + t2)**0.5
    return ret

  elif (func == 'gauss'):
    t1 = np.linalg.norm(x - mu)
    t1 = t1*t1
    t2 = 2*np.std(x)*np.std(x)

    ret = np.exp(-t1/t2)
    return ret

In [ ]:
def translate_to_rbf(x, mu, func):
  trans = []
  n, m = x.shape

  for i in range(n):
    d = []
    for j in range(k):
      dis = rad_basis_fn(x[i, :], mu[j, :], func)
      d.append(dis)
    trans.append(d)

  trans = np.array(trans).reshape(k, n)
  return trans

In [ ]:
def get_measures(y_pred, y):

  n, m = y_pred.shape
  for i in range(m):
    if (y_pred[0][i] >= 0.5):
      y_pred[0][i] = 1.0
    else:
      y_pred[0][i] = 0.0


  truepos = 0
  trueneg = 0
  falsepos = 0
  falseneg = 0

  for i in range(m):
    if (y[i][0] == 1.0):
      if (y_pred[0][i] == 1.0):
        truepos = truepos + 1
      else:
        falseneg = falseneg + 1

    else:
      if (y_pred[0][i] == 0.0):
        trueneg = trueneg + 1
      else:
        falsepos = falsepos + 1

  accuracy = (truepos + trueneg) / (truepos + trueneg + falsepos + falseneg)
  sensitivity = truepos / (truepos + falseneg)
  specificity = trueneg / (trueneg + falsepos)

  return accuracy, sensitivity, specificity

In [ ]:
func = 'gauss'
k = 3
iters = 500
kmeans = get_kmeans(k, iters)
k_for_kfold = 5


size = int(n / k_for_kfold)
start = 0
end = size


accuracy = []
sensitivity = []
specificity = []
weights = []


for kf in range(k_for_kfold):

  x = np.concatenate((X[:start, :], X[end:, :]), axis = 0)
  y = np.concatenate((Y[:start, :], Y[end:, :]), axis = 0)

  n, m = x.shape
  H = np.zeros((n, k))

  tx = X[start:end, :]
  ty = Y[start:end, :]

  for i in range(n):
    for j in range(k):
      p = rad_basis_fn(x[i:i+1, :], kmeans[j:j+1, :], func)
      H[i][j] = p


  Hinv = np.linalg.pinv(H)
  W = np.dot(Hinv, y)
  weights.append(W)

  x_trans = translate_to_rbf(tx, kmeans, func)
  y_trans = np.dot(W.T, x_trans)

  acc, sens, spec = get_measures(y_trans, ty)
  accuracy.append(acc)
  sensitivity.append(sens)
  specificity.append(spec)

  start += size
  end += size

In [ ]:
acc_final = max(accuracy)
spec_final = max(specificity)
sens_final = max(sensitivity)

acc_final, spec_final, sens_final

(1.0, 1.0, 1.0)